In [1]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.2 MB/s 
     |████████████████████████████████| 2.2 MB 49.4 MB/s 


In [36]:
import pandas as pd
import numpy as np

from math import floor
from sklearn.model_selection import train_test_split
from Levenshtein import distance as lev
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score, precision_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/My Drive/case_2/train.csv'

In [5]:
df = pd.read_csv(path, index_col='pair_id')

In [6]:
df.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [7]:
len(df[df['is_duplicate'] == 1])

3658

In [8]:
len(df[df['is_duplicate'] == 0])

494161

In [9]:
len(df[df['is_duplicate'] == 1]) / len(df) * 100

0.7348052203712594

In [10]:
features = df.drop('is_duplicate', axis=1)
target = df['is_duplicate']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=2022)

In [11]:
a = df['name_1'].to_list()

In [12]:
b = df['name_2'].to_list()

In [13]:
df_duplicate = df[df['is_duplicate'] == 1]
df_not_duplicate = df[df['is_duplicate'] == 0]

In [14]:
a_dub = df_duplicate['name_1'].to_list()
b_dub = df_duplicate['name_2'].to_list()

In [15]:
result = 0
list_result = []
for i, k in zip (a_dub, b_dub):
    s = lev(i, k)
    result += s
    list_result.append(s)

result = floor(result / len(a_dub))

In [16]:
print(result)

17


In [17]:
a_not_dub = df_not_duplicate['name_1'].to_list()
b_not_dub = df_not_duplicate['name_2'].to_list()

In [18]:
result = 0
n = 0
min_lev = 17
for i,k in zip (a_not_dub, b_not_dub):
    s = lev(i, k)
    result += s
    if s < min_lev:
        n += 1

result = round(result / len(a_not_dub), 4)

In [19]:
result

23.3951

In [20]:
n

123204

In [21]:
test_with_col = df[df['is_duplicate'] == 1].head(30).append(df[df['is_duplicate'] == 0].head(400)).reset_index()
test = test_with_col.drop(columns='is_duplicate', axis=0).reset_index()

In [23]:
t = test['name_1'].to_list()
t_1 = test['name_2'].to_list()

In [24]:
list_n = []
list_max = []
for i, k in zip (t, t_1):
    s = lev(i, k)
    maximum = max(len(i), len(k))
    list_n.append(s)
    list_max.append(maximum)

In [25]:
col = pd.DataFrame({'col':list_n})

In [26]:
test['col'] = list_n
test['max'] = list_max

In [27]:
test

,index,pair_id,name_1,name_2,col,max
0,0,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,26,62
1,1,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",16,27
2,2,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,19,29
3,3,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,34,38
4,4,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),20,28
...,...,...,...,...,...,...
425,425,397,Mjm Marketing Llc.,"T.A. Corporation Pvt., Ltd.",19,27
426,426,398,Dana,Sports Odessa Canada Inc.,22,25
427,427,399,Logistics Plus Canada Ltd.,Manuport Logistics Usa Llc,21,26
428,428,400,Bostik A/S,Lev TermoplAsticos Ltda,18,23


In [28]:
test['itog'] = (test['max'] - test['col']) / test['max']
test

,index,pair_id,name_1,name_2,col,max,itog
0,0,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,26,62,0.580645
1,1,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",16,27,0.407407
2,2,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,19,29,0.344828
3,3,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,34,38,0.105263
4,4,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),20,28,0.285714
...,...,...,...,...,...,...,...
425,425,397,Mjm Marketing Llc.,"T.A. Corporation Pvt., Ltd.",19,27,0.296296
426,426,398,Dana,Sports Odessa Canada Inc.,22,25,0.120000
427,427,399,Logistics Plus Canada Ltd.,Manuport Logistics Usa Llc,21,26,0.192308
428,428,400,Bostik A/S,Lev TermoplAsticos Ltda,18,23,0.217391


In [29]:
test['usl_30'] = (test['itog'] >= 0.3).astype('int')
test['usl_40'] = (test['itog'] >= 0.4).astype('int')
test['usl_50'] = (test['itog'] >= 0.5).astype('int')
test['usl_75'] = (test['itog'] >= 0.75).astype('int')
test['usl_90'] = (test['itog'] >= 0.90).astype('int')
test['usl_20'] = (test['itog'] >= 0.2).astype('int')

In [30]:
test

,index,pair_id,name_1,name_2,col,max,itog,usl_30,usl_40,usl_50,usl_75,usl_90,usl_20
0,0,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,26,62,0.580645,1,1,1,0,0,1
1,1,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",16,27,0.407407,1,1,0,0,0,1
2,2,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,19,29,0.344828,1,0,0,0,0,1
3,3,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,34,38,0.105263,0,0,0,0,0,0
4,4,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),20,28,0.285714,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,425,397,Mjm Marketing Llc.,"T.A. Corporation Pvt., Ltd.",19,27,0.296296,0,0,0,0,0,1
426,426,398,Dana,Sports Odessa Canada Inc.,22,25,0.120000,0,0,0,0,0,0
427,427,399,Logistics Plus Canada Ltd.,Manuport Logistics Usa Llc,21,26,0.192308,0,0,0,0,0,0
428,428,400,Bostik A/S,Lev TermoplAsticos Ltda,18,23,0.217391,0,0,0,0,0,1


In [31]:
test = test.merge(test_with_col['is_duplicate'].reset_index())

In [32]:
test

,index,pair_id,name_1,name_2,col,max,itog,usl_30,usl_40,usl_50,usl_75,usl_90,usl_20,is_duplicate
0,0,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,26,62,0.580645,1,1,1,0,0,1,1
1,1,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",16,27,0.407407,1,1,0,0,0,1,1
2,2,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,19,29,0.344828,1,0,0,0,0,1,1
3,3,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,34,38,0.105263,0,0,0,0,0,0,1
4,4,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),20,28,0.285714,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,425,397,Mjm Marketing Llc.,"T.A. Corporation Pvt., Ltd.",19,27,0.296296,0,0,0,0,0,1,0
426,426,398,Dana,Sports Odessa Canada Inc.,22,25,0.120000,0,0,0,0,0,0,0
427,427,399,Logistics Plus Canada Ltd.,Manuport Logistics Usa Llc,21,26,0.192308,0,0,0,0,0,0,0
428,428,400,Bostik A/S,Lev TermoplAsticos Ltda,18,23,0.217391,0,0,0,0,0,1,0


In [33]:
y_true = test['is_duplicate']
y_pred = test['usl_30']
f1_score(y_true, y_pred)

0.2631578947368421

In [37]:
recall_score(y_true, y_pred)

0.8333333333333334

In [38]:
precision_score(y_true, y_pred)

0.15625